### 1. INSTALL/IMPORT LIBRARIES

In [4]:
# Install Fred API library
%pip install fredapi > /dev/null

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Set matplotlib style
plt.style.use("fivethirtyeight")
palette = plt.rcParams["axes.prop_cycle"].by_key()["color"]

# Set a dark sns style:
styledic = {'figure.facecolor': 'gainsboro',
 'axes.labelcolor': '1',
 'xtick.direction': 'out',
 'ytick.direction': 'out',
 'xtick.color': '.15',
 'ytick.color': '.15',
 'axes.axisbelow': True,
 'grid.linestyle': '-',
 'text.color': '.15',
 'font.family': ['sans-serif'],
 'font.sans-serif': ['Arial',
  'DejaVu Sans',
  'Liberation Sans',
  'Bitstream Vera Sans',
  'sans-serif'],
 'lines.solid_capstyle': 'butt',
 'patch.edgecolor': 'b',
 'patch.force_edgecolor': True,
 'image.cmap': 'flare',
 'xtick.top': True,
 'ytick.right': False,
 'axes.grid': True,
 'axes.facecolor': '#d6d6f9',
 'axes.edgecolor': 'white',
 'grid.color': 'white',
 'axes.spines.left': True,
 'axes.spines.bottom': True,
 'axes.spines.right': True,
 'axes.spines.top': True,
 'xtick.bottom': True,
 'ytick.left': True}

sns.set_theme(style=styledic)
sns.set_context("notebook")
sns.set_palette("bright")

# Import FRED
from fredapi import Fred

# Set FRED API key
from config import FREDKEY as fk
print(f"Your fred API key is: {fk}")



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Your fred API key is: 592e1401c1b3dd13827a914e77312c39


### 2. CREATE FRED OBJECT

In [ ]:
fred = Fred(api_key=fk)

### 3. PULL UNEMPLOYMENT DATA

In [ ]:
# Search and pull results for "unemployment"
unemp_search = fred.search("unemployment", order_by="popularity", sort_order="desc")

# Check it out
print(f"Dataframe shape: {unemp_search.shape}")
unemp_search.head(3)

In [ ]:
# Get the unemplyment rate series
dfunemp = fred.get_series(series_id="UNRATE")

In [ ]:
# Print series
print("\nUSA unemployment % (monthly, since 1948):\n")
print(dfunemp)

# Plot the unemployment rate in seaborn
plt.figure(figsize=(12, 5))
sns.lineplot(data=dfunemp, lw=2)
plt.title("USA Unemployment Rate (1948-2024)", x=0.5, y=1.05)
plt.show()

# dfunemp.plot(figsize=(12, 5), title="Unemployment", lw=2)

In [ ]:
# Search and pull results for unemployment rate by state
unemp_search_df = fred.search("unemployment rate state", filter= ("frequency", "Monthly"))

# Check indesxes
print(f"\nResults for 'unemployment rate state':\n")
print(unemp_search_df.index)

# Query within that dataframe for the seasonally adjusted unemployment rate
print(f"\n Filtered by 'seasonally adjusted':\n")
unemp_df = unemp_search_df.query("seasonal_adjustment == 'Seasonally Adjusted' and units == 'Percent'")
print(unemp_df.index)

In [ ]:
# Get only those with "Unemployment Rate" in the title
unemp_df = unemp_df.loc[unemp_df["title"].str.contains("Unemployment Rate")]
unemp_df = unemp_df[unemp_df.index.str.endswith("UR")]
unemp_df.index

In [ ]:
unemp_df.title

In [ ]:
# take the field 'title' for each item,
# query the FRED API for series corresponding to each index code
# make eachs eries into a dataframe and assign it the previously extracted 'title'
# append the results to a list
# print lenght of list and one example

all_res = []

for i in unemp_df.index:
    title = unemp_df.loc[i, "title"]
    result = fred.get_series(i)
    result = result.to_frame(name=title)
    all_res.append(result)

print(len(all_res))
all_res[1]

In [ ]:
# concatenate all dataframes in the list into a single dataframe
concatdf = pd.concat(all_res, axis=1)

# take out the 'unemployment rate in ' part of the column names
concatdf.rename(columns=lambda x: x.replace('Unemployment Rate in ', ''), inplace=True)

# drop the columns referring to bigger-than-state areas and rename the dataframe to just DF
dropcolumns = concatdf.columns[concatdf.columns.str.contains('Census')]
df = concatdf.drop(columns=dropcolumns)

# check the resulting dataframe columns to see if they are all states
lista = df.columns.to_list()
print(len(lista))
print(lista)

In [ ]:
# Check whether there are na values
print(df.isna().sum(axis=0).all)

In [ ]:
# Check where the NA values are in Puerto Rico
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df["Puerto Rico"])

In [ ]:
nulls = df.loc[df["Puerto Rico"].isnull()]
print(nulls.index)
listanulls = nulls.index.to_list()
listanulls1 = []
for choclo in listanulls:
    x = str(choclo)
    x = x[0:-9:]
    print(x)
    listanulls1.append(x)
print(listanulls1)

In [ ]:
for date in listanulls1:
    df["Puerto Rico"].at[date] = 9.0

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df["Puerto Rico"])

In [ ]:
df.isnull().sum(axis=1).plot()

In [ ]:
px.line(df)

In [ ]:
fig_april = df.loc[df.index == "2020-04-01"].T.sort_values("2020-04-01", ascending=True) \
    .plot(kind="barh", figsize=(6,10), width=0.9, fontsize=8,
          title= "Unemployment by State, April 2020")

fig_april.legend().remove()
fig_april.set_xlabel("% unemployment")

plt.show()


como cambio el desempleo entre marzo y abril
rate de desempleo de abril - marzo


In [ ]:
df

In [ ]:
difference = df.loc['2020-04-01'] - df.loc['2020-03-01']
difference.sort_values(inplace=True)
dif_df = pd.DataFrame(difference)
dif_df.plot(kind="barh", figsize=(5,10), fontsize=6)

PASAR ESTO ULTIMO A VARIACION EXPRESADA EN PORCENTAJE
PULIR TODO EL NOTEBOOK